# Nested Sampling

Markov chain Monte Carlo (MCMC) methods of parameter estimation (for example) have been hugely successful in many areas where statistical analysis are of vital importance. However, there remain some issues. Some popular MCMC methods, like Hastings-Metropolis or Hamiltonian, run into problems with a multimodal posterior distribution, or one with large (curving) degeneracies between parameters. There is the additional "tweaking" of things such as "step size" and the long time required to converge to a solution for some problems.

Nested sampling (NS) is a concept originally proposed by Skilling (2004) which is targeted at the efficient calculation of the evidence, but also produces posterior inferences (and uncertainties) as a by-product. Graphically, an elliptical bound containing the current point set at each stage of the process is used to restrict the region around the posterior peak from which new samples are drawn. Multiple peaks in the parameter space can be detected and isolated, with separate ellipsoidal bounds constructed around each mode. Computational loads are reduced further by computing a mean and standard uncertainty in one sampling, eliminating the need for multiple runs. Depending upon the specifics of the problem an improvement over approaches like Hamiltonian Monte Carlo is that nested sampling does not require a differentiable phase space.

### Review
Bayesian inference provides a principled approach to determining a set of parameters $\Theta$ in a model (or hypothesis), $H$, for data, $\bf{D}$. Bayes theorum states that

$$Pr(\Theta \vert \bf{D},\it{H})=\frac{Pr(\bf{D} \vert \Theta,\it{H})Pr(\bf{\Theta} \vert \it{H})}{Pr(\bf{D} \vert \it{H})},$$

where $Pr(\Theta \vert \bf{D},\it{H}) \equiv P(\rm\Theta \vert \bf{D})$ is the posterior probability density of the model parameters, $Pr(\bf{D} \vert \Theta,\it{H}) \equiv \mathcal{L}(\rm\Theta)$ the likelihood of the data, and $Pr(\bf{\Theta} \vert \it{H}) \equiv \rm\pi(\Theta)$ the parameter prior. The final term, $Pr(\bf{D} \vert \it{H}) \equiv \mathcal{Z}$ (the Bayesian evidence), represents the factor required to normalize the posterior over the domain of $\Theta$ given by:

$$ \mathcal{Z}=\int_{\Omega_\Theta} \mathcal{L}(\Theta)\pi(\Theta)d(\Theta).$$

Model selection between two competing models, $H_0$ and $H_1$ can be achieved by comparing their respective posterior probabilities given the observed dataset as follows:

$$ R=\frac{Pr(H_1 \vert \bf{D})}{Pr(H_0 \vert \bf{D})}=\frac{Pr(\bf{D} \vert \it{H_1})Pr(H_1)}{Pr(\bf{D} \vert \it{H_0})Pr(H_0)}=\frac{\mathcal{Z_1}}{\mathcal{Z_0}}\frac{Pr(H_1)}{Pr(H_0)}.$$

Thus, Bayesian evidence plays a central role in Bayesian model selection.

As the average of the likelihood over the prior, the evidence is generally larger for a model if more of its parameter space is likely and smaller for a model with large areas in its parameter space having low likelihood values, even if the likelihood function is shaprly peaked. So, the evidence may be seen both as penalizing 'fine tuning' of a model against the observed data and as an automatic implementation of Occam's Razor.

### Nested Sampling

Nested sampling estimates the Bayesian evidence by transforming the multi-dimensional evidence integral over the prior density into a one-dimensional integral over an inverse survival function (with respect to prior mass) for the likelihood itself. This is accomplished by considering the survival function $X(\lambda)$, for $\mathcal{L}(\Theta)$, dubbed "the prior volume" here; namely,

$$ X(\lambda)=\int_{\Theta:\mathcal{L}(\Theta)>\lambda}\pi(\Theta)d(\Theta),$$

where the integral extends over the region(s) of parameter space contained within the iso-likelihood contour, $\mathcal{L}(\Theta)=\lambda$. Recalling that the expectation value of a non-negative random variable may be recovered by integration over its survival function (a result evident from integration by parts) we have (unconditionally):

$$ \mathcal{Z}=\int_{0}^{\infty} X(\lambda)d\lambda.$$

When $\mathcal{L}(X)$, the inverse of $X(\lambda)$, exists (i.e., when $\mathcal{L}(\Theta)$ is a continuous function with connected support) the evidence integral may thus be further rearranged as:

$$ \mathcal{Z}=\int_{0}^{1} \mathcal{L}(X)dX.$$

Now, if $\mathcal{L}(X)$ is known exactly (and Riemann integrable), be evaluating likelihoods, $\mathcal{L_i}=\mathcal{L}(X_i)$, for a deterministic sequence of $X$ values,

$$ 0<X_N< \ldots < X_2 < X_1 < X_0 = 1,$$

the evidence could in principle be approximated numerically using only standard quadrature methods as follows:

$$ \mathcal{Z} \approx \mathcal{\hat{Z}}=\sum_{i=1}^{N} \mathcal{L_i}\omega_i,$$

where the weights, $\omega_i$, for the simple trapezium rule are given by $\omega_i=\frac{1}{2}(X_{i-1}-X_{i+1})$. 

### Graphical representation

Let's take an aside from the discussion heavy in detailed lexicography. What is the basic interpretation of the method of nested sampling from a graphical perspective? What is the process that is implemented?

<img src="method.jpg">

Closing in on the best value of a parameter looks something like....

<img src="shrink.jpg">

Even for multiple peaks, say an overlapping point spread function in an image, the algorithm will close in on each separate peak, allowing for estimates of properties of each, no matter the conditions of the surface(s).

<img src="2D_example.jpg">

What about on multiple peaks or for multiple parameters, maybe even correlated parameters (posteriors) you wish to model? The NS algorithm will separate out each (in the simplest case) while progressing.

<img src="Multi_mode_example.jpg">

### Stopping criterion
The NS algorithm should terminate when the expected evidence contribution from the current set of live points is less than a user-defined tolerance. This expected remaining contribution can be estimated (cautiously) as $\Delta\mathcal{Z_i}=\mathcal{L_{max}}X_i$, where $\mathcal{L_{max}}$ is the maximum likelihood value amongst the current set of live points (with $X_i$ the expected value of remaining prior volume, as before).

### Posterior inferences

Although the NS algorithm is designed specifically to estimate the Bayesian evidence, inferences from the posterior distribution can be easily obtained using the final live points and the full sequence of discarded points from the NS process, i.e., the points with the lowest likelihood value at each iteration of the algorithm. Each such point is simply assigned the importance weight,

$$ p_i=\frac{\mathcal{L_i}\omega_i}{\sum_{j}\mathcal{L_i}\omega_i}=\frac{\mathcal{L_i}\omega_i}{\mathcal{\hat{Z}}},$$

from which sample-based estimates for the key posterior parameter summaries (e.g. means, standard deviations, covariances and so on) may be computed. 

### Application of Nested-Sampling

### Determination of the number of planets in the GJ667C system

Consider a series of models $H_{N_{obj}}$ , each with a fixed number of objects, i.e. with $N_{obj} = 0, 1, 2, . . . $, one then infers $N_{obs}$ by identifying the model with the largest marginal posterior probability $Pr(H_{N_{obj}} | D)$. Assuming that there are ${n_p}$ parameters per object and $n_n$ (nuisance) parameters common to all the objects, for $N_{obj}$ objects, there would be $N_{obj} n_p + n_n$ parameters to be inferred. Along with this increase in dimensionality, the complexity of the problem also increases with $N_{obj}$ due to the exponential increase in the number of modes as a result of counting degeneracy (there are $n!$ more modes for $N_{obj} = n$ than for $N_{obj} = 1$). If the contributions to the data from each object are reasonably
well separated and the correlations between parameters across objects are minimal, one can use the alternative approach of ‘single-source model’ by setting $N_{obj} = 1$, and therefore the model for the data consists of only a single object. This does not, however, restrict us to detecting only one object in the data. By modelling the data
in such a way, we would expect the posterior distribution to possess numerous peaks, each corresponding to the location of one of the objects. Consequently, the high dimensionality of the problem is traded with high multimodality in this approach which, depending on the statistical method employed for exploring the parameter space, could potentially simplify the problem enormously.

For $N_p$ planets and ignoring the planet–planet interactions, the RV at an instant $t_i$ observed at $j$th observatory can be calculated as:

$$\upsilon(t_i,j)=V_j-\sum^{N_p}_{p=1}K_p[sin(f_{i,p}+\varpi _p) +e_psin(\varpi _p)] $$
where

$V_j$ = systematic velocity with reference to $j$th observatory,

$K_p$ = velocity semi-amplitude of the pth planet,

$\varpi _p$ = longitude of periastron of the $p$th planet,

$f_{i,p}$ = true anomaly of the pth planet,

$e_p$ = orbital eccentricity of the pth planet, prior to the start of data taking, at which periastron occurred.


### Likelihood function

$$\mathcal{L(\bf {\Theta)}}=\frac{1}{2\pi \bf C}e^{[-\frac{1}{2}(\bf{\upsilon - \upsilon '})\bf{C}^{-1}(\bf{\upsilon - \upsilon '})]}$$

where


$$C[\upsilon(t_i,j),\upsilon(t_{i'},j')]=[{(\sigma_{i,j}/\alpha_j)}^2\delta_{ii'}+s^2_je^{[-|t_i-t_i'|/\tau_j]}]\delta_{jj'}$$

a covariance matrix with hyper-parameters that include the uncertainties of independent velocity measurements and a red noise component.


### Prior

$$Pr(\theta |H)=\frac{1}{(\theta+\theta_0)ln(1+\theta_{max}/\theta_0)}$$

behaves like a uniform prior for $\theta << \theta_0$ and like a Jeffreys prior
(uniform in log ) for $\theta << \theta_0$ . We set $K_0 = s_0 = 1~m s^{−1}$ and $K_{max} = 2129~m s^{−1}$ , which corresponds to a maximum planet–star mass ratio of 0.01.

### Results

Adopting the red noise model, we confirm the presence of planets GJ667Cb and c with periods 7.19 and 28.13 d,
respectively. There is some evidence for a third signal with orbital
period 91 d, but the planetary origins of this signal are doubtful.
We have also shown conclusively that erroneously adopting the
white noise model can result in detection of multiple further planets, which also explains the recent claims of the detection of up
to seven planets in this system. 

#### References
1. Original "Nested Sampling" reference (2004): http://adsabs.harvard.edu/abs/2004AIPC..735..395S
2. 2008 MNRAS, 384, 449 http://adsabs.harvard.edu/abs/2008MNRAS.384..449F
3. 2009 MNRAS, 398, 2049 http://adsabs.harvard.edu/abs/2009MNRAS.398.2049F
4. 2014, only on ArXiv, https://arxiv.org/abs/1306.2144
5. 2014, MNRAS, 437, 3540 http://adsabs.harvard.edu/abs/2014MNRAS.437.3540F
